In [94]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, VotingRegressor, IsolationForest
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression, ARDRegression
from xgboost import XGBRegressor
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor

from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK
from hyperopt.pyll import scope as ho_scope
from hyperopt.pyll.stochastic import sample as ho_sample

from sklearn.metrics import mean_squared_error as mse

In [47]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
def cleaner(df):
    colors = {'D' : 6, 'E' : 5, 'F' : 4, 'G' : 3, 'H' : 2, 'I' : 1, 'J' : 0}
    clarity = {'SI2' : 1, 'VVS2' : 5, 'VS2' : 3, 'VS1' : 4, 'SI1' : 2, 'VVS1' : 6, 'IF' : 7, 'I1' : 0}
    cut = {'Premium' : 4, 'Ideal' : 3, 'Very Good' : 2, 'Fair' : 0, 'Good' : 1}
    df['color'] = np.vectorize(colors.get)(df.color)
    df.clarity = np.vectorize(clarity.get)(df.clarity)
    df.cut = np.vectorize(cut.get)(df.cut)
    return df
train = cleaner(train)
test= cleaner(test)
X = train.drop(columns='price')
y = train.price
X_t= test

In [43]:
a = XGBRegressor(eta=0.1,max_depth=8,).fit(PolynomialFeatures().fit_transform(X_train), y_train)

[15:53:01] WARNING: ../src/learner.cc:573: 
Parameters: { "x_gamma" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [44]:
(-cross_val_score(a, PolynomialFeatures().fit_transform(X_train), y_train, scoring='neg_root_mean_squared_error', cv=5)).mean()

0.09145542664194659

In [45]:
from sklearn.model_selection import train_test_split

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)

In [37]:
xgbr=XGBRegressor()

In [110]:
space={
    
    'n_estimators': hp.quniform('n_estimators', 10, 1000, 25), # uniforme discreta de 10 a 1000 en saltos de 25
    
    'learning_rate': hp.uniform('learning_rate', 0.0001, 1.0), # uniforme continua entre 0.0001 y 1
    
    'max_depth': ho_scope.int(hp.quniform('my_param', 5, 10, q=1)),
    
    'gamma': hp.uniform('x_gamma', 0.1, 0.5)
}

In [111]:
def objetivo(space):
    
    modelo=XGBRegressor(n_estimators=int(space['n_estimators']),
                            learning_rate=space['learning_rate'],
                            gamma=space['gamma'],
                            max_depth = space['max_depth'],
                            objective='reg:squarederror'
                           )
    
    eval_set=[(PolynomialFeatures().fit_transform(X_train), y_train), (PolynomialFeatures().fit_transform(X_test), y_test)]
    
    modelo.fit(PolynomialFeatures().fit_transform(X_train), y_train, eval_set=eval_set, eval_metric='rmse', verbose=False)
    
    y_pred=modelo.predict(PolynomialFeatures().fit_transform(X_test))
    
    rmse=mse(y_test, y_pred)**0.5
    
    return {'loss': rmse, 'status': STATUS_OK}

In [112]:
mejor=fmin(fn=objetivo, space=space, algo=tpe.suggest, max_evals=10, trials=Trials())

mejor

100%|██████████████████████████████████████████| 10/10 [05:04<00:00, 30.45s/trial, best loss: 0.09353645881950222]


{'learning_rate': 0.03502550864700057,
 'my_param': 9.0,
 'n_estimators': 675.0,
 'x_gamma': 0.43533138808139427}

100%|██████████████████████████████████████████| 10/10 [05:18<00:00, 31.83s/trial, best loss: 0.09123348426331684]
{'learning_rate': 0.04029264786337595,
 'n_estimators': 500.0,
 'x_gamma': 0.14396489384644895}

In [ ]:
(-cross_val_score(a, PolynomialFeatures().fit_transform(X_train), y_train, scoring='neg_root_mean_squared_error', cv=5)).mean()